# tfdeepsurv for real data

## Introduction

Let's use `tfdeepsurv` package to build a neural network for predicting hazard ratio. This notebook 
will show you how to build and train a neural network.

## Preparation

For all things going well, you would be better to get acquainted with **Survival Analysis**. Otherwise, I suggest you read the [reference](https://lifelines.readthedocs.io/en/latest/Survival%20Analysis%20intro.html).

## Package installation

Please follow the instructions on [README](../README.md) to install `tfdeepsurv` package.

## Get it started

### Obtain datasets 

In [ ]:
import pandas as pd

train_data = pd.read_csv("data_train_filename.csv")
test_data = pd.read_csv("data_test_filename.csv")

train_data.head()

### Dataset statistics

In [ ]:
from tfdeepsurv.datasets import survival_stats

# specify the colnames of observed status and time in your dataset
colname_e = 'e'
colname_t = 't'

survival_stats(train_data, t_col=colname_t, e_col=colname_e, plot=True)

In [ ]:
survival_stats(test_data, t_col=colname_t, e_col=colname_e, plot=True)

### Survival data transfrom

The transformed survival data contains an new label. Negtive values are considered as right censored, 
and positive values are considered as event occurrence.

**NOTE**: In version 2.0, survival data must be transformed via `tfdeepsurv.datasets.survival_df`.

In [ ]:
from tfdeepsurv.datasets import survival_df

surv_train = survival_df(train_data, t_col=colname_t, e_col=colname_e, label_col="Y")
surv_test = survival_df(test_data, t_col=colname_t, e_col=colname_e, label_col="Y")

# columns 't' and 'e' are packed into an new column 'Y'
surv_train.head()

### Model initialization

**NOTE:** You can freely change all hyper-parameters during model initialization or training as you want.

All hyper-parameters is as follows:
- `nn_config`: model configuration
- `hidden_layers_nodes`: hidden layers configuration
- `num_steps`: training steps

Hyperparameters tuning can refer to README in directory `byopt`.

In [ ]:
from tfdeepsurv import dsnn

# Number of features in your dataset
input_nodes = len(surv_train.columns) - 1
# Specify your neural network structure
hidden_layers_nodes = [6, 3, 1]

# the arguments of dsnn can be obtained by Bayesian Hyperparameters Tuning.
# It would affect your model performance largely!
nn_config = {
    "learning_rate": 0.7,
    "learning_rate_decay": 1.0,
    "activation": 'relu', 
    "L1_reg": 3.4e-5, 
    "L2_reg": 8.8e-5, 
    "optimizer": 'sgd',
    "dropout_keep_prob": 1.0,
    "seed": 1
}

# ESSENTIAL STEP-1: Pass arguments
model = dsnn(
    input_nodes, 
    hidden_layers_nodes,
    nn_config
)

# ESSENTIAL STEP-2: Build Computation Graph
model.build_graph()

### Model training

You can save your trained model by passing `save_model="file_name.ckpt"` or load your trained model by passing `load_model="file_name.ckpt"`

In [ ]:
Y_col = ["Y"]
X_cols = [c for c in surv_train.columns if c not in Y_col]

# model saving and loading is also supported!
# read comments of `train()` function if necessary.

# ESSENTIAL STEP-3: Train Model
# `num_steps` is also a important parameters
watch_list = model.train(
    surv_train[X_cols], surv_train[Y_col],
    num_steps=1900,
    num_skip_steps=100,
    plot=True
)

### Model evaluation

In [ ]:
print("CI on training data:", model.evals(surv_train[X_cols], surv_train[Y_col]))
print("CI on test data:", model.evals(surv_test[X_cols], surv_test[Y_col]))

### Model prediction

Model prediction includes:
- predicting hazard ratio or log hazard ratio
- predicting survival function

In [ ]:
# predict log hazard ratio
print(model.predict(surv_test.loc[0:10, X_cols]))
# predict hazard ratio
print(model.predict(surv_test.loc[0:10, X_cols], output_margin=False))

In [ ]:
# predict survival function
model.predict_survival_function(surv_test.loc[0:5, X_cols], plot=True)

### tf.session close

To release resources, we use `model.close_session()` to close session in tensorflow!

In [ ]:
model.close_session()